In [1]:
#import dependencies
from bs4 import BeautifulSoup
import requests
import pandas as pd
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pymongo

# SQL Alchemy
from sqlalchemy import create_engine

# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()

# Config variables
from config import local_db_user, local_db_pwd, local_db_endpoint, local_db_port, local_db_name

In [2]:
# Create Engine and Pass in MySQL Connection

# local Postgres DB Connection  
local_engine = create_engine(f"postgresql://{local_db_user}:{local_db_pwd}@{local_db_endpoint}:{local_db_port}")
#print(f"postgresql://{local_db_user}:{local_db_pwd}@{local_db_endpoint}:{local_db_port}")

# Create a local database engine connection
local_conn = local_engine.connect()

# Create local database
try:
    local_conn.execution_options(isolation_level="AUTOCOMMIT").execute(f"CREATE DATABASE {local_db_name}")
except Exception as e:
    print(e)

# Connect to local database
try:
    local_engine = create_engine(f"postgresql://{local_db_user}:{local_db_pwd}@{local_db_endpoint}:{local_db_port}/{local_db_name}")
    local_conn = local_engine.connect()
except Exception as e:
    print(e)  
    
#confirm tables
local_engine.table_names()

(psycopg2.errors.DuplicateDatabase) database "project2_db" already exists

[SQL: CREATE DATABASE project2_db]
(Background on this error at: http://sqlalche.me/e/13/f405)


['2020_fantasy_stats']

In [3]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [C:\Users\Steven\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache


In [4]:
# URL of page to be scraped

#for season in range(2010, 20210):
season = '2020'
url = (f'https://fantasydata.com/nfl/fantasy-football-leaders?season={season}&seasontype=1&scope=1&subscope=1&startweek=1&endweek=1&aggregatescope=1&range=1')
url

# Retrieve page with the browser module
browser.visit(url)

# setup html parser
html = browser.html
fantasy_soup = BeautifulSoup(html, 'html.parser')

In [ ]:
# load 300 rows view
load_300 = browser.links.find_by_partial_text('300')
load_300.click()

In [6]:
# setup html parser
html = browser.html
fantasy_soup = BeautifulSoup(html, 'html.parser')

# load more rows
#load_more = browser.links.find_by_partial_text('Load More')
#load_more[1].click()
# for x in range(6):
#     load_more[1].click()

In [7]:
# save the stats table as a df
stats_table = fantasy_soup.find("table", id="stats_grid")
stats_df = pd.read_html(f'<table>${stats_table.tbody}</table>')[0]
stats_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,BUF,QB,16,4544,37,10,421,8,1,12,1,0.0,0,0,0,24.7,395.06
1,GB,QB,16,4299,48,5,149,3,1,-6,0,0.0,0,0,0,23.9,382.26
2,ARI,QB,16,3971,26,12,819,11,0,0,0,0.0,0,0,0,23.7,378.74
3,KC,QB,15,4740,38,6,308,2,0,0,0,0.0,0,0,0,25.0,374.40
4,HOU,QB,16,4823,33,7,444,3,0,0,0,0.0,0,0,0,23.1,369.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,KC,NT,15,0,0,0,0,0,0,0,0,7.5,0,2,0,6.1,91.80
296,LV,FS,13,0,0,0,0,0,0,0,0,0.0,2,0,0,7.0,91.50
297,CHI,WR,16,0,0,0,20,0,61,631,4,0.0,0,0,0,5.7,91.10
298,MIN,FS,16,0,0,0,0,0,0,0,0,0.0,0,0,0,5.7,91.00


In [ ]:
//*[@id="stats_grid"]/tbody/tr[52]/td[3]
//*[@id="stats_grid"]/tbody/tr[45]/td[4]
/html/body/div[6]/div/div[4]/div[2]/div/div[2]/div/div[2]/section/div[2]/div[3]/div[3]/div[2]/table/tbody/tr[48]

In [22]:
# save the names table as a df
table_names = fantasy_soup.find("div", class_="k-grid-content-locked")
names_df = pd.read_html(f'<table>${table_names.tbody}</table>')[0]
names_df

,0,1
0,1,Josh Allen
1,2,Aaron Rodgers
2,3,Kyler Murray
3,4,Patrick Mahomes
4,5,Deshaun Watson
...,...,...
295,296,Chris Jones
296,297,Johnathan Abram
297,298,Darnell Mooney
298,299,Anthony Harris


In [25]:
col_names = ['RK', 'NAME', 'TEAM', 'POS', 'GMS', 'PASS YDS', 'PASS TD', 'PASS INT', 'RUSH YDS', 'RUSH TD', 'REC', 'REC YDS', 'REC TD', 'DEF SCK', 'DEF INT', 'DEF FF', 'DEF FR', 'FPTS/G', 'FPTS']
combined_df = pd.concat([names_df, stats_df], axis=1)
combined_df.columns = col_names
combined_df

,RK,NAME,TEAM,POS,GMS,PASS YDS,PASS TD,PASS INT,RUSH YDS,RUSH TD,REC,REC YDS,REC TD,DEF SCK,DEF INT,DEF FF,DEF FR,FPTS/G,FPTS
0,1,Josh Allen,BUF,QB,16,4544,37,10,421,8,1,12,1,0.0,0,0,0,24.7,395.06
1,2,Aaron Rodgers,GB,QB,16,4299,48,5,149,3,1,-6,0,0.0,0,0,0,23.9,382.26
2,3,Kyler Murray,ARI,QB,16,3971,26,12,819,11,0,0,0,0.0,0,0,0,23.7,378.74
3,4,Patrick Mahomes,KC,QB,15,4740,38,6,308,2,0,0,0,0.0,0,0,0,25.0,374.40
4,5,Deshaun Watson,HOU,QB,16,4823,33,7,444,3,0,0,0,0.0,0,0,0,23.1,369.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,296,Chris Jones,KC,NT,15,0,0,0,0,0,0,0,0,7.5,0,2,0,6.1,91.80
296,297,Johnathan Abram,LV,FS,13,0,0,0,0,0,0,0,0,0.0,2,0,0,7.0,91.50
297,298,Darnell Mooney,CHI,WR,16,0,0,0,20,0,61,631,4,0.0,0,0,0,5.7,91.10
298,299,Anthony Harris,MIN,FS,16,0,0,0,0,0,0,0,0,0.0,0,0,0,5.7,91.00


In [26]:
combined_df.columns

Index(['RK', 'NAME', 'TEAM', 'POS', 'GMS', 'PASS YDS', 'PASS TD', 'PASS INT',
       'RUSH YDS', 'RUSH TD', 'REC', 'REC YDS', 'REC TD', 'DEF SCK', 'DEF INT',
       'DEF FF', 'DEF FR', 'FPTS/G', 'FPTS'],
      dtype='object')

In [18]:
#upload dataframe to database

combined_df.to_sql(name=f'2020_fantasy_stats', if_exists='replace', con=local_conn, index=False)
    

In [20]:
test = pd.read_sql('2020_fantasy_stats', local_conn)
print(len(test))
test.head()

300


,RK,NAME,TEAM,POS,GMS,PASS YDS,PASS TD,PASS INT,RUSH YDS,RUSH TD,REC,REC YDS,REC TD,DEF SCK,DEF INT,DEF FF,DEF FR,FPTS/G,FPTS
0,1,Josh Allen,BUF,QB,16,4544,37,10,421,8,1,12,1,0.0,0,0,0,24.7,395.06
1,2,Aaron Rodgers,GB,QB,16,4299,48,5,149,3,1,-6,0,0.0,0,0,0,23.9,382.26
2,3,Kyler Murray,ARI,QB,16,3971,26,12,819,11,0,0,0,0.0,0,0,0,23.7,378.74
3,4,Patrick Mahomes,KC,QB,15,4740,38,6,308,2,0,0,0,0.0,0,0,0,25.0,374.40
4,5,Deshaun Watson,HOU,QB,16,4823,33,7,444,3,0,0,0,0.0,0,0,0,23.1,369.32
